# Reprogramming Tracker

### 1.0 Import libraries

In [ ]:
try:
    import cv2
except ImportError:
    !pip install opencv-python

try:
    import cellpose
except ImportError:
    !pip install cellpose

from os import listdir
from os.path import isfile, join
import imgfileutils as imf
from core import *
from io import BytesIO
from PIL import Image
from cellpose import utils, models, io, plot
import skimage.io
import numpy as np
from skimage.measure import regionprops
from math import pi
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm
from plotly.subplots import make_subplots
from skimage.filters import threshold_otsu, threshold_local

### 2.0 Create dictionary

In [ ]:
dct = {}

### 3.0 Get .CZI names

In [ ]:
path = '/volumes/emilio_passport/msc_neuro/lab/rep_tracker/data/d6Ngn2/d6Ngn2_V2/'     # Insert path to the folder with the CZI file
path_fig = '/volumes/emilio_passport/msc_neuro/lab/rep_tracker/figures/d6Ngn2_Bleb/ppt/'     # Path to save generated images

no_files = input('Single File or Multiple Files (S/M)? ')

if no_files.lower() == 's':
    ### --- File Name
    for f in listdir(path): 
        if isfile(join(path, f)) and f.lower().endswith('.czi'):
            print(f)
    filename = input('Insert name of file + format (i.e.: file.czi): ')
    dct[filename] = {}
    dct[filename]['filename'] = filename
    try:
        ### --- Get the CZI object using the path + filename
        dct[filename]['file_path'] = path + dct[filename]['filename']
    except:
        print('Invalid Filename!')
elif no_files.lower() == 'm':
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f)) and f.lower().endswith('.czi')]
    try:
        onlyfiles.remove('.DS_Store')
    except:
        pass
    for file in onlyfiles:
        dct[file] = {}
        ### --- Get the CZI object using the path + filename
        dct[file]['filename'] = file
        dct[file]['file_path'] = path + dct[file]['filename']

### 4.0 Load .CZI files and metadata

In [ ]:
### --- Read metadata and array differently for OME-TIFF or CZI data
for file in dct:
    if dct[file]['filename'].lower().endswith('.ome.tiff') or dct[file]['filename'].lower().endswith('.ome.tif'):

        ### --- Return value is an array of order (T, Z, C, X, Y)
        (dct[file]['array'], dct[file]['omexml']) = io.read_ometiff(dct[file]['file_path'])
        dct[file]['metadata'], dct[file]['add_metadata'] = imf.get_metadata(dct[file]['file_path'], series = 0)

    if dct[file]['filename'].lower().endswith('.czi'):
        print(file)
        ### --- Get the array and the metadata
        dct[file]['array'], dct[file]['metadata'], dct[file]['add_metadata'] = imf.get_array_czi(dct[file]['file_path'], 
                                                                                                 return_addmd = False)

### 4.1 Obtain shapes and dimensions of array and metadata (Optional)

In [ ]:
flag = True
for file in dct:
    if flag == True:
        ### --- Shape of the numpy array
        print('Array Shape: ', dct[file]['array'].shape)

        ### --- Dimension order of the metadata
        print('Dimension Order (BioFormats) : ', dct[file]['metadata']['DimOrder BF Array'])

        ### --- Shape and dimension entry of the CZI file as returned by czifile.py
        print('CZI Array Shape : ', dct[file]['metadata']['Shape'])
        print('CZI Dimension Entry : ', dct[file]['metadata']['Axes'])
        flag = False

### 4.2 Show all the metadata (Optional)

In [ ]:
flag = True
for file in dct:
    if flag == True:
        for key, value in dct[file]['metadata'].items():
            print(key, ' : ', value)     # print all key-value pairs for the dictionary
        flag = False

### 5.0 Select working channels

In [ ]:
flag = True
for file in dct:
    ### --- Identify the content of each channel
    dct[file]['channels'] = {}
    
    dct[file]['working_array'] = wk_array(dct[file]['array'], dct[file]['metadata']['Axes'])
    
    for channel in range(len(dct[file]['metadata']['Channels'])):
        dct[file]['channels']['ch%s' % str(channel)] = dct[file]['working_array'][channel]
        
    if flag == True:
        channel_nuclei, channel_induction_reporter, channel_reprogrammed, channel_gfap, __ = identify_channels(dct[file]['metadata']['Channels'])
    flag = False

### 6.0 Segmentation of Nuclei

In [ ]:
method = input('Insert method for segmentation (cellpose OR classic): ')

if method.lower() == 'classic':
    
    thresh_options = ['K-means', 'Otsu', 'Isodata', 'Li', 'Mean', 'Minimum',
                      'Triangle', 'Yen', 'Sauvola', 'Adaptive_Otsu']
    
    for option in thresh_options:
        print(option)
        
    thresh_option = input('Insert desired thresholding method: ')
    
for file in tqdm(dct):
    for channel in dct[file]['channels']:
        if channel == 'ch%s' % channel_nuclei:
            if method.lower() == 'cellpose':
                dct[file]['masks_%s' % channel], dct[file]['flows_%s' % channel] = segment_image_nuclei(
                    dct[file]['channels'][channel], method = method, diameter = 30)
            elif method.lower() == 'classic':
                dct[file]['masks_%s' % channel] = segment_image_nuclei(dct[file]['channels'][channel], 
                                                                       method = method, 
                                                                       thresh_option = thresh_option, 
                                                                       name = dct[file]['filename'], 
                                                                       h_fraction = 0.1)

### 6.1 Show nuclear segmentation (Optional)

In [ ]:
for file in dct:
    for channel in dct[file]['channels']:
        if channel == 'ch%s' % channel_nuclei:
            if method.lower() == 'cellpose':
                channels = [0, 0]
                fig = plt.figure(figsize = (12,5))
                plot.show_segmentation(fig, dct[file]['channels'][channel], dct[file]['masks_%s' % channel], 
                                       dct[file]['flows_%s' % channel][0], channels = channels)
                plt.tight_layout()
                plt.show()
            elif method.lower() == 'classic':
                plt.rcParams['figure.figsize'] = [30/2.54, 22.5/2.54]
                plt.imshow(dct[file]['channels'][channel], 'gray')
                masked = np.ma.masked_where(dct[file]['masks_%s' % channel] <= 0, dct[file]['masks_%s' % channel])
                plt.imshow(masked, alpha = 0.8)
                plt.title(dct[file]['filename'])
                plt.show()

### 7.0 Obtain first set of nuclear parameters

In [ ]:
for file in dct:
    for channel in dct[file]['channels']:
        if channel == 'ch%s' % channel_nuclei:
            cell_no = []
            nuclei_area = []
            nuclei_intensity = []
            circularities = []
            nuclei_perimeter = []
            eccentricities = []
            solidities = []
            major_axes = []
            minor_axes = []
            axes_ratios = []
            xpos = []
            ypos = []
            props = regionprops(dct[file]['masks_%s' % channel], intensity_image = dct[file]['channels'][channel])
            for p in props:
                if p['label'] > 0:
                    
                    cell_no.append(p['label'])
                    
                    area = p['area'] * (float(dct[file]['metadata']['XScale']) * float(dct[file]['metadata']['YScale']))
                    nuclei_area.append(round(area))
                    
                    nuclei_intensity.append(round(p['mean_intensity']))
                    
                    perimeter = p['perimeter'] * dct[file]['metadata']['XScale']
                    nuclei_perimeter.append(round(p['perimeter']))
                    
                    circularity = 4 * pi * (area / perimeter ** 2)
                    circularities.append(round(circularity, 3))
                    
                    eccentricities.append(round(p['eccentricity'], 3))
                    
                    solidities.append(round(p['solidity'], 3))
                    
                    major_axis = p['major_axis_length'] * dct[file]['metadata']['XScale']
                    major_axes.append(round(major_axis, 1))
                    
                    minor_axis = p['minor_axis_length'] * dct[file]['metadata']['XScale']
                    minor_axes.append(round(minor_axis, 1))                  

                    axes_ratio = minor_axis / major_axis
                    axes_ratios.append(round(axes_ratio, 3))
                    
                    cY, cX = p['centroid']
                    xpos.append(round(cX))
                    ypos.append(round(cY))
                    
    dct_df = {'cell_no': cell_no, 'nucleus_intensity': nuclei_intensity, 'nucleus_area': nuclei_area, 
             'nucleus_perimeter': nuclei_perimeter, 'major_axis': major_axes, 'minor_axis': minor_axes,
             'axes_ratio': axes_ratios, 'circularity': circularities, 'eccentricity': eccentricities,
             'solidity': solidities, 'x_pos': xpos, 'y_pos': ypos}

    dct[file]['df'] = pd.DataFrame(data = dct_df)
    
    fig = go.Figure(go.Table(header = dict(values = ['Cell<br>Number', 'Nuclear<br>Intensity', 'Nuclear<br>Area<br>(µm2)', 
                                                     'Nucleus<br>Perimeter<br>(µm)', 'Major<br>Axis<br>(µm)', 
                                                     'Minor<br>Axis<br>(µm)',
                                                     'Axes<br>Ratio', 'Circularity', 'Eccentricity', 
                                                     'Solidity', 'x pos', 'y pos'], 
                                           align = 'left'), 
                             cells = dict(values = [dct[file]['df'][k].tolist() for k in dct[file]['df'].columns[0:]], align = 'left')))
    fig.update_layout(title = dct[file]['filename'].split('.')[0])
    fig.show()

### 7.1 Show distribution of nuclear area and intensity

In [ ]:
x_nuc_area = []
x_int = []

for file in dct:
    for v in dct[file]['df']['nucleus_area']:
        x_nuc_area.append(v)
    for v in dct[file]['df']['nucleus_intensity']:
        x_int.append(v)
        
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True)
fig.add_trace(go.Box(x = x_nuc_area, boxpoints = 'suspectedoutliers', fillcolor = 'rgba(7,40,89,0.5)', 
                     line_color = 'rgb(7,40,89)', showlegend = False, name = ''), row = 1, col = 1)
fig.add_trace(go.Histogram(x = x_nuc_area, histnorm = 'probability', marker_color = 'rgb(7,40,89)',
                          name = 'Area', showlegend = False), row = 2, col = 1)
fig.update_layout(title = "Distribution of Cells' Nucleus Area", 
                  xaxis = dict(autorange = True, showgrid = True, zeroline = True, gridwidth=1), width = 1000, 
                  height = 400, template = "plotly_white")
fig.show()
#fig.write_image(path_fig + 'area_hist_bp.pdf')


fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True)
fig.add_trace(go.Box(x = x_int, boxpoints = 'suspectedoutliers', fillcolor = 'rgba(8,81,156,0.5)', 
                     line_color = 'rgb(8,81,156)', showlegend = False, name = ''), row = 1, col = 1)
fig.add_trace(go.Histogram(x = x_int, histnorm = 'probability', marker_color = 'rgb(8,81,156)',
                          name = 'Intensity', showlegend = False), row = 2, col = 1)
fig.update_layout(title = "Distribution of Cells' Nuclei Intensity", 
                  xaxis = dict(autorange = True, showgrid = True, zeroline = True, gridwidth=1), width = 1000, 
                  height = 400, template = "plotly_white")
fig.show()
#fig.write_image(path_fig + 'intensity_hist_bp.pdf')

### 8.0 Apply filters to data

In [ ]:
max_area_filter = input('Enter an UPPER threshold for nuclei area (µm2): ')
min_area_filter = input('Enter a LOWER threshold for nuclei area (µm2): ')
circularity_filter = input('Enter an LOWER threshold for nuclei circularity: ')
intensity_filter = input('Enter an LOWER threshold for nuclei intensity: ')

for file in dct:
    dct[file]['df_filtered'] = dct[file]['df']
    to_be_removed = dct[file]['df_filtered'].index[dct[file]['df_filtered']['nucleus_area'] > int(max_area_filter)].tolist()
    to_be_removed += dct[file]['df_filtered'].index[dct[file]['df_filtered']['nucleus_area'] < int(min_area_filter)].tolist()
    to_be_removed += dct[file]['df_filtered'].index[dct[file]['df_filtered']['circularity'] < float(circularity_filter)].tolist()
    to_be_removed += dct[file]['df_filtered'].index[dct[file]['df_filtered']['nucleus_intensity'] < int(intensity_filter)].tolist()
    to_be_removed = np.array(to_be_removed)
    to_be_removed = np.unique(to_be_removed).tolist()
    dct[file]['df_filtered'] = dct[file]['df_filtered'].drop(to_be_removed)
    dct[file]['df_filtered'].reset_index(drop = True, inplace = True)

    fig = go.Figure(go.Table(header = dict(values = ['Cell<br>Number', 'Nuclear<br>Intensity', 'Nuclear<br>Area<br>(µm2)', 
                                                     'Nucleus<br>Perimeter<br>(µm)', 'Major<br>Axis<br>(µm)', 
                                                     'Minor<br>Axis<br>(µm)',
                                                     'Axes<br>Ratio', 'Circularity', 'Eccentricity', 
                                                     'Solidity', 'x pos', 'y pos'], 
                                           align = 'left'), 
                             cells = dict(values = [dct[file]['df_filtered'][k].tolist() for k in dct[file]['df_filtered'].columns[0:]], 
                                          align = 'left')))
    fig.update_layout(title = dct[file]['filename'].split('.')[0])
    fig.show()

### 8.1 Show new data distribution

In [ ]:
flag = True
for file in dct:
    if flag == False:
        temp_df = temp_df.append(dct[file]['df_filtered'])
    if flag == True:
        temp_df = dct[file]['df_filtered'].copy()
        flag = False
    
fig = px.density_heatmap(temp_df, x = 'nucleus_area', y = 'nucleus_intensity', marginal_x = 'histogram', 
                         marginal_y = 'histogram', color_continuous_scale = px.colors.diverging.RdYlBu_r,
                         nbinsx = 15, nbinsy = 15)
fig.show()
#fig.write_image(path_fig + 'intensityVSarea_hm.pdf')

### 9.0 Image thresholding for other channels

In [ ]:
thresh_options = ['K-means', 'Otsu', 'Isodata', 'Li', 'Mean', 'Minimum',
                  'Triangle', 'Yen', 'Sauvola', 'Adaptive_Otsu']

for option in thresh_options:
    print(option)

thresh_option = input('Insert desired thresholding method: ')
    
for file in dct:
    for channel in dct[file]['channels']:
        if channel != 'ch%s' % channel_nuclei:
            dct[file]['thresh_%s' % (channel)] = thresholding(dct[file]['channels'][channel], dct[file]['filename'],
                                                             thresh_option)

### 9.1 Generate binary images of the other channels

In [ ]:
for file in dct:
    for channel in dct[file]['channels']:
        if channel != 'ch%s' % channel_nuclei:
            dct[file]['closing_%s' % (channel)] = binary_img(dct[file]['thresh_%s' % (channel)], 
                                                             dct[file]['filename'])

### 10.0 Identify cell type

In [ ]:
for file in dct:
    for channel in dct[file]['channels']:
        if channel == 'ch%s' % channel_nuclei:
            cell_type = assign_celltype(df = dct[file]['df_filtered'], 
                                        binary_rfp = dct[file]['closing_ch%s' % (channel_induction_reporter)], 
                                        binary_b3 = dct[file]['closing_ch%s' % (channel_reprogrammed)], 
                                        mask_nuclei = dct[file]['masks_%s' % channel], 
                                        col_cells = 'cell_no')
            dct_df = {'cell_type': cell_type}
            df_add = pd.DataFrame(data = dct_df)
            dct[file]['df_celltype'] = pd.concat([dct[file]['df_filtered'], df_add], axis = 1, sort = True)
            fig = go.Figure(go.Table(header = dict(values = ['Cell<br>Number', 'Nuclear<br>Intensity', 'Nuclear<br>Area<br>(µm2)', 
                                                             'Nucleus<br>Perimeter<br>(µm)', 'Major<br>Axis<br>(µm)', 
                                                             'Minor<br>Axis<br>(µm)',
                                                             'Axes<br>Ratio', 'Circularity', 'Eccentricity', 
                                                             'Solidity', 'x pos', 'y pos', 'Cell<br>Type'], 
                                                   align = 'left'), 
                                     cells = dict(values = [dct[file]['df_celltype'][k].tolist() for k in dct[file]['df_celltype'].columns[0:]], 
                                                  align = 'left')))
            fig.update_layout(title = dct[file]['filename'].split('.')[0])
            fig.show()            

### 11.0 Obtain second set of nuclear parameters

In [ ]:
for file in dct:
    hc_no = []
    hc_inner = []
    hc_outer = []
    hc_edge = []
    intensity_inner = []
    intensity_outer = []
    intensity_edge = []
    intensity_rfp = []
    intensity_b3 = []
    sum_int_b3_edges = []
    intensity_gfap = []
    sum_int_nuclei = []
    sum_int_inners = []
    sum_int_outers = []
    sum_int_edges = []
    for channel in dct[file]['channels']:
        if channel == 'ch%s' % channel_nuclei:
            for cell in tqdm(dct[file]['df_celltype']['cell_no']):
                dots, dots_inner, dots_outer, dots_edge = find_hcdots(dct[file]['channels'][channel], 
                                                                      dct[file]['masks_%s' % channel], 
                                                                      cell_no = cell, 
                                                                      xscale = float(dct[file]['metadata']['XScale']))
                hc_no.append(dots)
                hc_inner.append(dots_inner)
                hc_outer.append(dots_outer)
                hc_edge.append(dots_edge)
                
                int_inner, int_outer, int_edge, __ = find_intesities_layers(dct[file]['channels'][channel], 
                                                                           dct[file]['masks_%s' % channel], 
                                                                           cell_no = cell, 
                                                                           xscale = float(dct[file]['metadata']['XScale']))
                intensity_inner.append(int_inner)
                intensity_outer.append(int_outer)
                intensity_edge.append(int_edge)
                
                sum_int_nucleus, sum_int_inner, sum_int_outer, sum_int_edge = sum_intensities(dct[file]['channels'][channel], 
                                                                                              dct[file]['masks_%s' % channel], 
                                                                                              cell_no = cell,
                                                                                              xscale = float(dct[file]['metadata']['XScale']))
                
                sum_int_nuclei.append(sum_int_nucleus)               
                sum_int_inners.append(sum_int_inner)               
                sum_int_outers.append(sum_int_outer)               
                sum_int_edges.append(sum_int_edge)
                
                int_rfp = find_intensity(dct[file]['channels']['ch%s' % (channel_induction_reporter)], 
                                         dct[file]['masks_%s' % channel], 
                                         cell_no = cell)
                intensity_rfp.append(int_rfp)

                int_b3 = find_intensity(dct[file]['channels']['ch%s' % (channel_reprogrammed)], 
                                         dct[file]['masks_%s' % channel], 
                                         cell_no = cell)
                intensity_b3.append(int_b3)

                __, __, __, sum_int_edge_b3 = sum_intensities(dct[file]['channels']['ch%s' % (channel_reprogrammed)], 
                                                                                              dct[file]['masks_%s' % channel], 
                                                                                              cell_no = cell,
                                                                                              xscale = float(dct[file]['metadata']['XScale']))
                
                sum_int_b3_edges.append(sum_int_edge_b3)
                
                __, __, __, int_gfap = find_intesities_layers(dct[file]['channels']['ch%s' % (channel_gfap)], 
                                                              dct[file]['masks_%s' % channel],
                                                              cell_no = cell,
                                                              xscale = float(dct[file]['metadata']['XScale']))
                intensity_gfap.append(int_gfap)
                
            dct_df = {'hc_no': hc_no, 'hc_inner': hc_inner, 'hc_outer': hc_outer, 'hc_edge': hc_edge,
                     'intensity_inner': intensity_inner, 'intensity_outer': intensity_outer, 
                     'intensity_edge': intensity_edge, 'sum_int_nucleus': sum_int_nuclei,
                      'sum_int_inner': sum_int_inners, 'sum_int_outer': sum_int_outers,
                      'sum_int_edge': sum_int_edges, 'intensity_rfp': intensity_rfp, 
                     'intensity_b3': intensity_b3, 'sum_int_b3_edges': sum_int_b3_edges,
                      'intensity_gfap': intensity_gfap}
            
            df_add = pd.DataFrame(data = dct_df)
            dct[file]['df_final'] = pd.concat([dct[file]['df_celltype'], df_add], axis = 1, sort = True)
            
            fig = go.Figure(go.Table(header = dict(values = ['Cell<br>Number', 'Cell<br>Type', 'Hc Dots', 
                                                            'Hc Dots<br>Inner', 'Hc Dots<br>Outer',
                                                            'Hc Dots<br>Edge', 'Int<br>Inner', 'Int<br>Outer',
                                                            'Int<br>Edge', 'Sum<br>Int<br>Nucleus',
                                                             'Sum<br>Int<br>Inner','Sum<br>Int<br>Outer',
                                                             'Sum<br>Int<br>Edge','Int<br>RFP', 'Int<br>B3',
                                                             'Sum<br>Int<br>B3<br>Edge',
                                                             'Int<br>GFAP'], 
                                                   align = 'left'), 
                                     cells = dict(values = [dct[file]['df_final'][k].tolist() for k in dct[file]['df_final'].columns[np.r_[0, 12:27]]], 
                                                  align = 'left')))
            fig.update_layout(title = dct[file]['filename'].split('.')[0])
            fig.show()            
                

In [ ]:
for file in dct:
    dct[file]['df_final']['source'] = [str(file.split('.')[0]) for n in range(len(dct[file]['df_final']))]

### 12.0 Save data

In [ ]:
import scipy.io
from pathlib import Path

if len(dct) == 1:
    out_file = filename.split('.')[0] + '.csv'    # Change name of output file
    command = input('Do you want to create NEW file or APPEND to existing file (new/append)? ')

    my_file = Path(path + out_file)

    try:
        if command.lower() == 'new':
            df_final.to_csv(my_file)
        elif command.lower() == 'append':
            if my_file.is_file():
                df_final.to_csv(my_file, mode = 'a', header = False)
            else:
                df_final.to_csv(my_file)
        filename_save = filename.split('.')[0]
        scipy.io.savemat(path + filename_save + '_masks.mat', mdict = {'out': dct['markers_ch%s' % channel_nuclei]})
        scipy.io.savemat(path + filename_save + '_nuclei.mat', mdict = {'out': channels['ch%s' % channel_nuclei]})
        print('Successfully saved!')
    except:
        print('Ops! An error occurred...')
elif len(dct) > 1:
    save_s_m = input('Save as Single or Multiple Files (S/M)? ')
    if save_s_m.lower() == 'm':
        for file in dct:
            out_file = dct[file]['filename'].split('.')[0] + '.csv'    # Change name of output file

            my_file = Path(path + out_file)

            try:
                dct[file]['df_final'].to_csv(my_file)
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_binary_gfap.mat', mdict = {'out': dct[file]['closing_ch%s' % channel_gfap]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_gfap.mat', mdict = {'out': dct[file]['channels']['ch%s' % channel_gfap]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_binary_rfp.mat', mdict = {'out': dct[file]['closing_ch%s' % channel_induction_reporter]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_rfp.mat', mdict = {'out': dct[file]['channels']['ch%s' % channel_induction_reporter]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_binary_b3.mat', mdict = {'out': dct[file]['closing_ch%s' % channel_reprogrammed]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_b3.mat', mdict = {'out': dct[file]['channels']['ch%s' % channel_reprogrammed]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_masks.mat', mdict = {'out': dct[file]['masks_ch%s' % channel_nuclei]})
                scipy.io.savemat(path + dct[file]['filename'].split('.')[0] + '_nuclei.mat', mdict = {'out': dct[file]['channels']['ch%s' % channel_nuclei]})
                print(dct[file]['filename'].split('.')[0], 'Successfully saved!')
            except:
                print('Ops! An error occurred...')